In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode (no UI)
options.add_argument('--disable-gpu')  # Useful for headless mode
options.add_argument('--no-sandbox')  # Bypass OS security model
options.add_argument('--disable-dev-shm-usage')  # Overcome limited resources

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = 'https://sa.ucla.edu/RCO/public/search?q=tech'
driver.get(url)

time.sleep(5)


iframes = driver.find_elements(By.TAG_NAME, "iframe")
print(f"Found {len(iframes)} iframes.")
if iframes:
    driver.switch_to.frame(iframes[0])  

#used for dynamic content to allow all clubs to load before parsing 
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Allow time for new content to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'card') or contains(@class, 'row')]"))
    )
except:
    print("Club elements did not load in time.")
    print(driver.page_source)  # Debugging: Print the page source
    driver.quit()
    exit()

#create club containers that include these elements 
club_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'card') or contains(@class, 'row') or contains(@class, 'coldmd-4') or contains(@href, 'mailto:') or contains(@href, 'instagram.com') or contains(@class, 'col-md-2')]")

all_clubs_name = []
all_clubs_insta = []
all_clubs_email = []
all_clubs_data = []
all_clubs_signature = []
count = 1
signcount =1

for club in club_containers: 
    signature = "(.//div[contains(@class, 'col-md-2')])" + "[" + str(signcount) + "]"
    try:
        signature_tag = driver.find_element(By.XPATH, signature )
        signature_name = signature_tag.text.strip() if signature_tag else "none" 
    except:
        signature_name = None
    if signature_name == "Signatory 1:" or signature_name == "Signatory 2:" or signature_name == "Signatory 3:": 
        next_signature = "(.//div[contains(@class, 'col-md-2')])" + "[" + str(signcount + 1) + "]"
        next_signature_tag = driver.find_element(By.XPATH, next_signature)
        next_signature_name = next_signature_tag.text.strip() if next_signature_tag else "none"
        all_clubs_signature.append(next_signature_name) 
    signcount +=1 
    
for club in club_containers:

    #parses elements based on instance count 
    name_parameter = "(//div[starts-with(@class, 'row bold')])" + "[" + str(count) + "]" 
    email = "(.//a[starts-with(@href, 'mailto:')])" + "[" + str(count) + "]" 
    instagram = "(.//a[contains(@href, 'instagram.com')])" + "[" + str(count) + "]" 
    count +=1

    



    try:
        name_tag = driver.find_element(By.XPATH, name_parameter)
        club_name= name_tag.text.strip() if name_tag else "No name found"
    except:
        club_name = None
        
    try:
        email_tag = driver.find_element(By.XPATH, email)
        club_email = email_tag.text.strip() if email_tag else "No email found"
    except:
        club_email = None



    try:
        instagram_tag = driver.find_element(By.XPATH, instagram)
        club_instagram = instagram_tag.get_attribute("href") if instagram_tag else "No Instagram found"
    except:
        club_instagram = None

    if club_instagram != None: all_clubs_insta.append(club_instagram)
    #print(iteration)
    #print(firstinsta)
    #print(club_instagram)
    if (club_instagram == None and club_email == None and club_name == None):
        continue
    if (club_name != None):
        all_clubs_name.append(club_name)
    if (club_email != None): 
        all_clubs_email.append(club_email.split("@")[0])

    #fix instagram handles later but pain in the ass 
    

#print(temp_insta)

scount = 0
for i in range(len(all_clubs_name)):
    #print (i)
    #print(all_clubs_insta[i])
    #print(all_clubs_name[i])
    club_data = { 
        'name' : all_clubs_name[i], 
        'email' : all_clubs_email[i], 
        'signatory' : all_clubs_signature[scount: scount + 3],
    }
    all_clubs_data.append(club_data)
    scount += 3

driver.quit()

print(all_clubs_insta) 
    
for club in all_clubs_data: 
    print(club)
#print(all_clubs_insta)
#print(all_clubs_name)
#print(all_clubs_email)


Found 0 iframes.
['https://www.instagram.com/accessucla?igsh=MWQ1ZGUxMzBkMA==', 'https://www.instagram.com/acm.teachla/', 'https://www.instagram.com/aiandeyeucla', 'https://www.instagram.com/airesucla/', 'https://www.instagram.com/aamsatucla/', 'https://www.instagram.com/ucla_aises/', 'https://www.instagram.com/aiaa_ucla/', 'https://www.instagram.com/UCLA_ASIG', 'https://www.instagram.com/art.ificial_ucla/?igsh=NTc4MTIwNjQ2YQ%3D%3D&utm_source=qr', 'https://www.instagram.com/ascend_ucla/', 'https://www.instagram.com/asmebruins/', 'https://www.instagram.com/acm.ucla/', 'https://www.instagram.com/uclaacmw', 'https://www.instagram.com/uclaaurum/', 'https://www.instagram.com/balletcompanyatucla', 'https://www.instagram.com/uclabiokind/', 'https://www.instagram.com/bmesucla/', 'https://www.instagram.com/blockchainucla/', 'https://www.instagram.com/bruin_ai/', 'https://instagram.com/bruinbodybuilding', 'https://www.instagram.com/bruinburlesque/', 'https://www.instagram.com/bruincapital/?hl=en